In [4]:
import glob
import os
import MeCab
import sys
import re
from collections import Counter

##### ファイルをリストに格納 #####
texts = []
dir_name_txt = "non_death_txt"
os.chdir('data/'+dir_name_txt)
name = glob.glob("*") #ファイル名リスト
for i in range(len(name)):
    f = open(str(name[i]), encoding="utf-8")
    texts.append(f.read())  # ファイル終端まで全て読んだデータを返す
    f.close()
os.chdir('..')
os.chdir('..')
print(len(name),len(texts))

data = []
num_test = 10 # testデータの数
for i in range(len(name)):
    ##### 名詞をリストに格納 #####
    items = []
    # パース
    mecab = MeCab.Tagger()
    parse = mecab.parse(texts[i])
    lines = parse.split('\n')
    items = (re.split('[\t,]', line) for line in lines)
    

    # 名詞をリストに格納
    words = [item[0]
             for item in items
             if (item[0] not in ('EOS', '', 't', 'ー') and
                 item[1] == '名詞' and item[2] == '一般')]

    # 名詞の間にスペースを入れて1文にする
    data_ = ""
    for j in range(len(words)):
        data_ = data_ + " "+words[j]
    data.append(data_)
    #print(len(data))

hanrei_test = {"file_name":[],"data":[],"target_name":[],"target":[]}
for i in range(num_test):
    hanrei_test["file_name"].append(name[i])
    hanrei_test["data"].append(data[i])
    hanrei_test["target_name"].append("non_death")
    hanrei_test["target"].append(0)

hanrei_train = {"file_name":[],"data":[],"target_name":[],"target":[]}
for i in range(num_test,len(data)):
    hanrei_train["file_name"].append(name[i])
    hanrei_train["data"].append(data[i])
    hanrei_train["target_name"].append("non_death")
    hanrei_train["target"].append(0)
    #{"data":str(i), "target_name":str(i), "target":str(i)})
print(len(hanrei_test["data"]),len(hanrei_train["data"]))

#-----------------------死刑判例文(1)--------------------------#

##### ファイルをリストに格納 #####
texts = []
dir_name_txt = "death_txt"
os.chdir('data/'+dir_name_txt)
name = glob.glob("*")
for i in range(len(name)):
    f = open(str(name[i]), encoding="utf-8")
    texts.append(f.read())  # ファイル終端まで全て読んだデータを返す
    f.close()
os.chdir('..')
os.chdir('..')
#print(len(name),len(texts))

data = []
for i in range(len(name)):
    ##### 名詞をリストに格納 #####
    items = []
    # パース
    mecab = MeCab.Tagger()
    parse = mecab.parse(texts[i])
    #print(parse)
    lines = parse.split('\n')
    items = (re.split('[\t,]', line) for line in lines)
    #print(items)
    # 名詞をリストに格納
    words = [item[0]
             for item in items
             if (item[0] not in ('EOS', '', 't', 'ー') and
                 item[1] == '名詞' and item[2] == '一般')]

    #名詞の間にスペースを入れて1文にする
    data_ = ""
    for j in range(len(words)):
        data_ = data_ + " "+words[j]
        
    data.append(data_)
#print(data)

#print(len(data))
for i in range(num_test):
    hanrei_test["file_name"].append(name[i])
    hanrei_test["data"].append(data[i])
    hanrei_test["target_name"].append("death")
    hanrei_test["target"].append(1)

for i in range(num_test,len(data)):
    hanrei_train["file_name"].append(name[i])
    hanrei_train["data"].append(data[i])
    hanrei_train["target_name"].append("death")
    hanrei_train["target"].append(1)
    

print(len(hanrei_test["data"]),len(hanrei_train["data"]))
#print(hanrei_train["target_name"])

54 54
10 44
20 75


In [ ]:
## create word2vec
import logging
import numpy as np
from gensim.models import Word2Vec
import MeCab
import time
from sklearn.preprocessing import normalize
import sys
import re

start = time.time()
tokenizer =  MeCab.Tagger("-Owakati")  
sentences = []
print ("Parsing sentences from training set...")

# Loop over each news article.
for review in tqdm(df["news"]):
    try:
        # Split a review into parsed sentences.
        result = tokenizer.parse(review).replace("\u3000","").replace("\n","")
        result = re.sub(r'[0123456789０１２３４５６７８９！＠＃＄％＾＆\-|\\＊\“（）＿■×※⇒—●(：〜＋=)／*&^%$#@!~`){}…\[\]\"\'\”:;<>?＜＞？、。・,./『』【】「」→←○]+', "", result)
        h = result.split(" ")
        h = list(filter(("").__ne__, h))
        sentences.append(h)
    except:
        continue

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

num_features = 200     # Word vector dimensionality
min_word_count = 20   # Minimum word count
num_workers = 40       # Number of threads to run in parallel
context = 10          # Context window size
downsampling = 1e-3   # Downsample setting for frequent words

print ("Training Word2Vec model...")
# Train Word2Vec model.
model = Word2Vec(sentences, workers=num_workers, hs = 0, sg = 1, negative = 10, iter = 25,\
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling, seed=1)

model_name = str(num_features) + "features_" + str(min_word_count) + "minwords_" + str(context) + "context_len2alldata"
model.init_sims(replace=True)
# Save Word2Vec model.
print ("Saving Word2Vec model...")
model.save("../japanese-dataset/livedoor-news-corpus/model/"+model_name)
endmodeltime = time.time()

print ("time : ", endmodeltime-start)